# Requirements

Requieres [Ollama](https://ollama.com/) with the [llama3](https://ollama.com/library/llama3) model to be installed

In [1]:
# Required Packages

# !pip install streamlit
# !pip install ollama
# !pip install langchain
# !pip install langchain-chroma
# !pip install langchain-community
# !pip install langchain-ollama
# !pip install chromadb

# Load Webpage

In [2]:
from langchain_community.document_loaders import WebBaseLoader

webpage_url = "https://en.wikipedia.org/wiki/Kempten_University_of_Applied_Sciences"
loader = WebBaseLoader(webpage_url)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

# Vector data base

In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(splits, embeddings)

# RAG chain

In [6]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
def retrieve_context(question):
    retriever = vector_store.as_retriever(k=5)
    retrieved_docs = retriever.invoke(question)
    return combine_docs(retrieved_docs)

In [8]:
import ollama

def ollama_llm(question, context):
    response = ollama.chat(model='llama3', messages=[
        {
            'role': 'system', 
            'content': f"Answer the following question based on the context from {webpage_url}:\n\n{context}"
        },
        {
            'role': 'user', 
            'content': question
        }])
    return response['message']['content']

In [9]:
question = "When was Kempten University of Applied Sciences founded?"

context = retrieve_context(question)
print(context)

Faculties[edit]
The Kempten University of Applied Sciences has the following faculties:

Business Administration
Electrical Engineering
Computer Science
Mechanical Engineering
Social and Health Studies
Tourism Management
Cooperations[edit]
The Kempten University of Applied Sciences is part of the Internationale Bodensee-Hochschule. Besides there are 80 Universities outside of Germany which have a cooperation with the Kempten University of Applied Sciences.

References[edit]

Kempten University of Applied SciencesHochschule KemptenGebäude der Hochschule KemptenTypePublicEstablished1977PresidentRobert F. SchmidtAcademic staffabout 300[1]Students4,644[1]LocationKempten im Allgäu, Bavaria, Germany47°42′57″N 10°18′49″E﻿ / ﻿47.7158333333°N 10.3136111111°E﻿ / 47.7158333333; 10.3136111111Websitewww.hochschule-kempten.de
The Kempten University of Applied Sciences or Hochschule Kempten is a university of applied sciences in Kempten im Allgäu, Germany.

Retrieved from "https://en.wikipedia.org/w/

In [10]:
response = ollama_llm(question, context)
print(response)

According to the provided Wikipedia article, the Kempten University of Applied Sciences was established in 1977.
